# RAG-chatbot för handbollsregler

### Imports

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### PDF to text

In [4]:
from pypdf import PdfReader

def pdf_to_text(path: str) -> str:
    reader = PdfReader(path)
    pages = []
    for page in reader.pages:
        pages.append(page.extract_text() or "")
    return "\n".join(pages)

pdf_path = "PDF/spelregler_handboll_2025.pdf"
text = pdf_to_text(pdf_path)

print(text[:500])  # Print the first 500 characters of the extracted text
print("\nTotal characters extracted:", len(text))

 
3  
 
 
 
 
 
 
 
 
 
 
 
SPELREGLER 
Med kommentarer för 
HANDBOLL 
➢ Klarlägganden till spelreglerna 
 
➢ Reglemente rörande avbytarområdet 
 
➢ Reglemente gällande Shoot-Out 
 
➢ PM för domare, tidtagare, sekreterare 
och föreningar 
 
 
 
Utgivna av 
SVENSKA HANDBOLLFÖRBUNDET 
 
Gällande från den 1 juli 2025 
 
 
Eftertryck utan tillstånd är förbjudet 

 
4  
 
 
 
 
INNEHÅLLSFÖRTECKNING 
Regel 1: Spelplanen ..................................................................................

Total characters extracted: 187336


### Chunkning

In [61]:
def chunk_text(text, chunk_size=900, overlap=150):
    chunks = []
    i = 0
    
    while i < len(text):
        chunk = text[i:i + chunk_size]
        chunks.append(chunk)
        i += chunk_size - overlap
    
    return chunks

chunks = chunk_text(text, chunk_size=900, overlap=150)

print("Chunks:", len(chunks))
print("Max length:", max(len(c) for c in chunks))


Chunks: 250
Max length: 900


In [15]:
lengths = [len(c) for c in chunks]

print("Max chunk length:", max(lengths))
print("Average chunk length:", sum(lengths) / len(lengths))


Max chunk length: 900
Average chunk length: 898.744


In [16]:
biggest_idx = np.argmax(lengths)
print("Biggest chunk index:", biggest_idx)
print(chunks[biggest_idx][:1500])


Biggest chunk index: 0
 
3  
 
 
 
 
 
 
 
 
 
 
 
SPELREGLER 
Med kommentarer för 
HANDBOLL 
➢ Klarlägganden till spelreglerna 
 
➢ Reglemente rörande avbytarområdet 
 
➢ Reglemente gällande Shoot-Out 
 
➢ PM för domare, tidtagare, sekreterare 
och föreningar 
 
 
 
Utgivna av 
SVENSKA HANDBOLLFÖRBUNDET 
 
Gällande från den 1 juli 2025 
 
 
Eftertryck utan tillstånd är förbjudet 

 
4  
 
 
 
 
INNEHÅLLSFÖRTECKNING 
Regel 1: Spelplanen .............................................................................................. 5 
Regel 2: Speltiden, slutsignal och timeout .................................................... 10 
Regel 3: Bollen................................................................................................... 14 
Regel 4: Laget, byte av spelare, utrustning .................................................... 15 
Regel 5: Målvakten..............................................


In [17]:
# Tiktoken - OpenAI:s tokenizer
import tiktoken

EMBED_MODEL = "text-embedding-3-small" 

enc = tiktoken.encoding_for_model(EMBED_MODEL)

for i, c in enumerate(chunks):
    tokens = len(enc.encode(c))
    if tokens > 7500:
        print("Too large chunk:", i, tokens)

### Embeddings

In [14]:
import numpy as np

EMBED_MODEL = "text-embedding-3-small"

def embed_text(texts):
    embeddings = []
    
    for i, text in enumerate(texts):
        if i % 10 == 0:
            print(f"Embedding chunk {i+1}/{len(texts)}")

        response = client.embeddings.create(
            model=EMBED_MODEL,
            input=text
        )

        embeddings.append(response.data[0].embedding)

    return np.array(embeddings, dtype="float32")

print("Total chunks:", len(chunks))
embeddings = embed_text(chunks)
print("Embeddings shape:", embeddings.shape)

Total chunks: 250
Embedding chunk 1/250
Embedding chunk 11/250
Embedding chunk 21/250
Embedding chunk 31/250
Embedding chunk 41/250
Embedding chunk 51/250
Embedding chunk 61/250
Embedding chunk 71/250
Embedding chunk 81/250
Embedding chunk 91/250
Embedding chunk 101/250
Embedding chunk 111/250
Embedding chunk 121/250
Embedding chunk 131/250
Embedding chunk 141/250
Embedding chunk 151/250
Embedding chunk 161/250
Embedding chunk 171/250
Embedding chunk 181/250
Embedding chunk 191/250
Embedding chunk 201/250
Embedding chunk 211/250
Embedding chunk 221/250
Embedding chunk 231/250
Embedding chunk 241/250
Embeddings shape: (250, 1536)


### FAISS Index 

In [18]:
import faiss

# embedding dimension
dim = embeddings.shape[1]

# normalize embeddings to unit length
faiss.normalize_L2(embeddings)

index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print("Index size:", index.ntotal)

Index size: 250


### Retrieval function

In [20]:
def retrieve(query: str, top_k: int = 5):
    """
    Hämtar de mest relevanta chunks från vektordatabasen baserat på en fråga.
    
    Args:
        query: Frågan/sökningen du vill göra
        top_k: Hur många relevanta chunks du vill få tillbaka (standard: 5)
    
    Returns:
        Lista med (index, relevans-score, chunk-text) för de mest relevanta chunks
    """
    # 1. Skapa en embedding för frågan
    query_embedding = client.embeddings.create(
        model=EMBED_MODEL,
        input=query
    ).data[0].embedding

    # 2. Konvertera till numpy array och normalisera
    query_embedding = np.array([query_embedding], dtype="float32")
    faiss.normalize_L2(query_embedding)

    # 3. Sök i FAISS-indexet efter de mest lika chunks
    scores, ids = index.search(query_embedding, top_k)

    # 4. Formatera resultaten
    hits = []
    for i, score in zip(ids[0], scores[0]):
        hits.append((int(i), float(score), chunks[int(i)])) 
    return hits

# Testa funktionen med en exempelfråga
print("=== TEST AV RETRIEVE-FUNKTIONEN ===\n")
test_query = "Vad är reglerna för passning?"
results = retrieve(test_query, top_k=3)

print(f"Fråga: '{test_query}'\n")
print(f"Hittade {len(results)} relevanta chunks:\n")

for idx, (chunk_id, score, chunk_text) in enumerate(results, 1):
    print(f"--- Resultat {idx} ---")
    print(f"Chunk ID: {chunk_id}")
    print(f"Relevans-score: {score:.4f}")
    print(f"Text (första 200 tecken): {chunk_text[:200]}...")
    print()

=== TEST AV RETRIEVE-FUNKTIONEN ===

Fråga: 'Vad är reglerna för passning?'

Hittade 3 relevanta chunks:

--- Resultat 1 ---
Chunk ID: 151
Relevans-score: 0.6503
Text (första 200 tecken): Om ett försök till passning inte kan kontrolleras beroende på 
en regelförseelse av försvarande spelare.  
• Om ett försök till passning styrs ut över sidlinjen eller den yttre 
mållinjen av en försva...

--- Resultat 2 ---
Chunk ID: 203
Relevans-score: 0.6428
Text (första 200 tecken): illåtet Undantag för målvakt 
Sockor  Samma färg  
 
Klädsel för ledare 
 Skall vara civil- eller sportklädsel och färgen 
ska skilja sig från motståndarlagets 
utespelartröjor. 
 
För ytterligare det...

--- Resultat 3 ---
Chunk ID: 150
Relevans-score: 0.6329
Text (första 200 tecken): a passningsvägar för spelaren med bollen blockeras aktivt 
av aktivt försvarsspel; 
• spelaren med bollen vänder sig bort från målet. 
Generellt gäller detta när drivet i spelet, tempot i det uppställ...



### RAG 

In [46]:
CHAT_MODEL = "gpt-4o-mini"

def answer(query: str, top_k: int = 5, temperature: float = 0.4, debug: bool = True):
    hits = retrieve(query, top_k=top_k)

    context = "\n\n---\n\n".join([chunk for _idx, _score, chunk in hits])

    system_message = (
        "Du är en hjälpsam kompis som kan handbollsregler. "
        "Svara på svenska med enkel, avslappnad ton. "
        "Svara kort först (1–2 meningar), och lägg sen en lite längre förklaring om det behövs. "
        "Om du inte vet svaret, säg det är okänt istället för att hitta på något. "
        "Använd bara information från KONTEXT. "
        "Om svaret inte finns i KONTEXT: säg 'Jag hittar inget om det i dokumentet.'"
    )

    user_message = (
        f"FRÅGA:\n{query}\n\n"
        f"KONTEXT:\n{context}\n\n"
        "SVAR:"
    )

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
    )

    if debug:
        print("=== SOURCES ===")
        for idx, score, _ in hits:
            print(f"chunk {idx} | score {score:.3f}")
    
    return resp.choices[0].message.content


In [47]:
# Test

print(answer("Hur många steg får man ta i handboll?", top_k=9, temperature=0.4))

=== SOURCES ===
chunk 46 | score 0.645
chunk 45 | score 0.635
chunk 43 | score 0.617
chunk 222 | score 0.598
chunk 21 | score 0.587
chunk 44 | score 0.581
chunk 22 | score 0.581
chunk 23 | score 0.570
chunk 215 | score 0.562
Man får ta högst tre steg i handboll. Efter att ha tagit emot bollen måste spelaren spela den inom tre sekunder eller ta tre steg.

Det är viktigt att notera att stegen räknas efter att spelaren har fått kontroll över bollen. Om spelaren fångar bollen när han eller hon inte har kontakt med golvet, räknas det inte som ett steg att sätta ner fötterna. Det är också tillåtet att studsa bollen, vilket ger spelaren möjlighet att röra sig mer fritt.


### Save index + chunks

In [48]:
import pickle
faiss.write_index(index, "handboll_faiss.index")

with open("chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

### Load

In [49]:
# Load saved index and chunks to verify it works

index = faiss.read_index("handboll_faiss.index")

with open("chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

In [50]:
test_questions = [
    "Hur många steg får man ta i handboll?",
    "Vad är passivt spel?",
    "Vad händer vid rött kort?",
    "Vad är ett frikast?"
]

for q in test_questions:
    print("Q:", q)
    print(answer(q, debug=False))
    print("------")


Q: Hur många steg får man ta i handboll?
Man får ta högst tre steg i handboll efter att man har tagit emot bollen. Om man har bollen under kontroll måste den spelas inom tre sekunder eller efter tre steg. 

Det innebär att när en spelare fångar bollen, får de max ta tre steg innan de måste passa eller skjuta. Om spelaren dribblar, kan de fortsätta dribbla utan att räkna stegen, men så fort de stoppar dribblingen måste de följa regeln om tre steg eller tre sekunder.
------
Q: Vad är passivt spel?
Passivt spel är när ett lag inte aktivt försöker skapa målchanser och istället spelar långsamt eller taktiskt för att vinna tid. Det kan leda till att domarna dömer laget för att de inte är tillräckligt offensiva.

I handboll syftar reglerna om passivt spel till att göra spelet mer dynamiskt och mindre händelselöst. Det handlar om att förhindra att spelare drar ut på tiden genom att långsamt flytta bollen eller undvika att attackera. Om domarna bedömer att ett lag spelar passivt, kan de ge en v

### Chat memory

In [53]:
chat_history = []

def chat(query: str, top_k: int = 5, temperature: float = 0.4, debug: bool = True):
    global chat_history
    
    # Make retrieval based on the current query
    hits = retrieve(query, top_k=top_k)
    context = "\n\n---\n\n".join([chunk for _idx, _score, chunk in hits])

    system_message = (
        "Du är en hjälpsam kompis som kan handbollsregler. "
        "Svara på svenska med enkel, avslappnad ton. "
        "Svara kort först (1–2 meningar), och lägg sen en lite längre förklaring om det behövs. "
        "Om du inte vet svaret, säg det är okänt istället för att hitta på något. "
        "Använd bara information från KONTEXT. "
        "Om svaret inte finns i KONTEXT: säg 'Jag hittar inget om det i dokumentet.'"
    )

    # Build the user message with context
    user_message = (
        f"FRÅGA:\n{query}\n\n"
        f"KONTEXT:\n{context}\n\n"
        "SVAR:"
    )

    # Build messages: system + history + new query
    messages = [{"role": "system", "content": system_message}]
    messages += chat_history[-6:]  # last 6 messages
    messages.append({"role": "user", "content": user_message})

    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=temperature,
        messages=messages
    )
    answer_text = resp.choices[0].message.content

    # Update chat history (answer + question, not the whole context)
    chat_history.append({"role": "user", "content": query})
    chat_history.append({"role": "assistant", "content": answer_text})

    if debug:
        print("=== SOURCES ===")
        for idx, score, chunk in hits:
            print(f"chunk {idx} | score {score:.3f} | {chunk[:120].replace(chr(10),' ')}...")
        print()

    return answer_text

### Test

In [55]:
print(chat("Hur många steg får man ta i handboll?", top_k=6))
print(chat("Gäller det om man studsar bollen också?", top_k=6))
print(chat("Vad händer om man studsar bollen igen efter att man tagit tre steg?", top_k=6))    

=== SOURCES ===
chunk 46 | score 0.645 | fter fattar bollen med en hand eller med båda händerna, måste den   23      spelas efter högst tre (3) steg eller inom t...
chunk 45 | score 0.635 | ttas från en plats till en annan och den andra foten  ”släpas” bakom, räknas det inte som ett steg. Det är i enlighet me...
chunk 43 | score 0.617 | ortfarande i spel.  22                Regel 7 – Hur bollen får spelas, passivt spel      Spela bollen    Bollen anses va...
chunk 222 | score 0.598 | en lämnat utespelarens hand får målvakten röra sig framåt.  Ingen målvakt får dock lämna sin målgård**). Målvakt i anfal...
chunk 21 | score 0.587 | n lag-timeout på en (1) minut i varje halvlek av den  ordinarie speltiden (men inte i förlängningar). Detaljerade instru...
chunk 44 | score 0.581 | en  medan hans fötter inte har kontakt med golvet så räknas inte att sätta ned en fot eller  båda samtidigt som ett steg...

Man får ta högst tre steg i handboll efter att ha tagit emot bollen.

När en spelare har

### Add PDF documents

In [59]:
# List all PDF files we want to include
import os

pdf_folder = "PDF"
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

print("PDF-files in folder:")
for pdf in pdf_files:
    print(f"  - {pdf}")

PDF-files in folder:
  - ihf_rules.pdf
  - spelregler_handboll_2025.pdf


In [60]:
# Debug: kontrollera att funktionerna inte har blivit överskrivna
print("pdf_to_text type:", type(pdf_to_text))
print("chunk_text type:", type(chunk_text))

pdf_to_text type: <class 'function'>
chunk_text type: <class 'str'>


In [62]:


def process_all_pdfs(pdf_folder: str, chunk_size: int = 900, overlap: int = 150):
    """
    Reads all PDFs and creates chunks with metadata.
    Returns:
        - all_chunks: list of chunk texts
        - chunk_metadata: list of dicts containing source and language
    """
    all_chunks = []
    chunk_metadata = []
    
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        print(f"\nProcessing: {pdf_file}")
        
        # Read text
        text = pdf_to_text(pdf_path)
        print(f"   Extracted {len(text)} characters")
        
        # Determine language based on filename (simple heuristic)
        if "ihf" in pdf_file.lower() or "rules" in pdf_file.lower():
            language = "en"
        else:
            language = "sv"
        
        # Chunk the text
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
        print(f"   Created {len(chunks)} chunks (language: {language})")
        
        # Add chunks and metadata
        for chunk in chunks:
            all_chunks.append(chunk)
            chunk_metadata.append({
                "source": pdf_file,
                "language": language
            })
    
    return all_chunks, chunk_metadata

# Run the function
all_chunks, chunk_metadata = process_all_pdfs("PDF", chunk_size=900, overlap=150)

print(f"\nTotal chunks: {len(all_chunks)}")
print(f"   Number of sources: {len(set(m['source'] for m in chunk_metadata))}")


Processing: ihf_rules.pdf
   Extracted 185210 characters
   Created 247 chunks (language: en)

Processing: spelregler_handboll_2025.pdf
   Extracted 187336 characters
   Created 250 chunks (language: sv)

Total chunks: 497
   Number of sources: 2


In [63]:
# Create embeddings for all chunks

print(f"Skapar embeddings för {len(all_chunks)} chunks...")

all_embeddings = embed_text(all_chunks)

print(f"\nEmbeddings shape: {all_embeddings.shape}")

Skapar embeddings för 497 chunks...
Embedding chunk 1/497
Embedding chunk 11/497
Embedding chunk 21/497
Embedding chunk 31/497
Embedding chunk 41/497
Embedding chunk 51/497
Embedding chunk 61/497
Embedding chunk 71/497
Embedding chunk 81/497
Embedding chunk 91/497
Embedding chunk 101/497
Embedding chunk 111/497
Embedding chunk 121/497
Embedding chunk 131/497
Embedding chunk 141/497
Embedding chunk 151/497
Embedding chunk 161/497
Embedding chunk 171/497
Embedding chunk 181/497
Embedding chunk 191/497
Embedding chunk 201/497
Embedding chunk 211/497
Embedding chunk 221/497
Embedding chunk 231/497
Embedding chunk 241/497
Embedding chunk 251/497
Embedding chunk 261/497
Embedding chunk 271/497
Embedding chunk 281/497
Embedding chunk 291/497
Embedding chunk 301/497
Embedding chunk 311/497
Embedding chunk 321/497
Embedding chunk 331/497
Embedding chunk 341/497
Embedding chunk 351/497
Embedding chunk 361/497
Embedding chunk 371/497
Embedding chunk 381/497
Embedding chunk 391/497
Embedding chunk

In [64]:
dim = all_embeddings.shape[1]

# Normalize
faiss.normalize_L2(all_embeddings)

# Create index
new_index = faiss.IndexFlatIP(dim)
new_index.add(all_embeddings)

print(f"New index created with {new_index.ntotal} vectors")

New index created with 497 vectors


In [65]:
# Test retrieval with both sources

def retrieve_with_sources(query: str, top_k: int = 5):
    """Retrieve with metadata about source"""
    query_embedding = client.embeddings.create(
        model=EMBED_MODEL,
        input=query
    ).data[0].embedding

    query_embedding = np.array([query_embedding], dtype="float32")
    faiss.normalize_L2(query_embedding)

    scores, ids = new_index.search(query_embedding, top_k)

    hits = []
    for i, score in zip(ids[0], scores[0]):
        hits.append({
            "chunk_id": int(i),
            "score": float(score),
            "text": all_chunks[int(i)],
            "source": chunk_metadata[int(i)]["source"],
            "language": chunk_metadata[int(i)]["language"]
        })
    return hits

# Test with a Swedish query
test_query = "Vad händer vid rött kort?"
print(f"Searching: '{test_query}'\n")

results = retrieve_with_sources(test_query, top_k=5)

for i, hit in enumerate(results, 1):
    print(f"--- Result {i} ---")
    print(f"Source: {hit['source']} ({hit['language']})")
    print(f"Relevance: {hit['score']:.4f}")
    print(f"Text: {hit['text'][:200]}...")
    print()

Searching: 'Vad händer vid rött kort?'

--- Result 1 ---
Source: spelregler_handboll_2025.pdf (sv)
Relevance: 0.5621
Text: edelbart informeras efter beslutet. 
 
Vid detta tillfälle ska domaren även visa det blå kortet efter att det röda visats, 
detta som en information. 
 
Mer än en ojusthet vid samma tillfälle  
16:9 O...

--- Result 2 ---
Source: spelregler_handboll_2025.pdf (sv)
Relevance: 0.5588
Text: genom att hålla upp ett gult kort (domartecken 13 ) 
 
Utvisning 
 
16:3 En utvisning (2 minuter) är den tillämpliga bestraffningen: 
a) för ett felaktigt byte om en ytterligare spelare beträder spelp...

--- Result 3 ---
Source: spelregler_handboll_2025.pdf (sv)
Relevance: 0.5277
Text:  
IHF:s Domartecken 
 
När ett inkast eller frikast dömes, ska domarna omedelbart visa riktningen för det 
följande kastet (tecken 7 eller 9). 
Därefter bör i förekommande fall det tillämpliga obligat...

--- Result 4 ---
Source: spelregler_handboll_2025.pdf (sv)
Relevance: 0.5163
Text: portsligt upp

In [66]:
# Save FAISS index
faiss.write_index(new_index, "handboll_faiss.index")

# Save chunks AND metadata together
data_to_save = {
    "chunks": all_chunks,
    "metadata": chunk_metadata
}

with open("chunks_with_metadata.pkl", "wb") as f:
    pickle.dump(data_to_save, f)

# Also save only chunks for backward compatibility with the app
with open("chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

print("Saved:")
print("   - handboll_faiss.index")
print("   - chunks_with_metadata.pkl (with source info)")
print("   - chunks.pkl (for backward compatibility)")

Saved:
   - handboll_faiss.index
   - chunks_with_metadata.pkl (with source info)
   - chunks.pkl (for backward compatibility)
